# Prototype

In [156]:
import numpy as np

### Gameplay

In [157]:
class Gameplay:
    def __init__(self, n_rows, n_cols):
        self.columns = n_cols
        self.rows = n_rows
        self.rules = []
        self.tiles = np.zeros((n_rows, n_cols), dtype=object)
        for i in range(n_rows):
            for j in range(n_cols):
                self.tiles[i,j] = Tile()

        self.prefixes = ['baba', 'rock', 'water', 'skull', 'wall', 'flag']
        self.suffixes = ['you', 'push', 'sink', 'defeat', 'stop', 'win']
            

    def get_rules(self):
        self.rules = []
    
        for i in range(self.rows):
            for j in range(self.columns):
                if self.tiles[i,j].find_word() == "is":
                    print("yes")
                    #left -> right
                    if j-1 >= 0 and j+1 <=self.columns-1:
                        if self.tiles[i,j-1].find_word() in self.prefixes and self.tiles[i,j+1].find_word() in self.suffixes:
                            rule = Rule(self.tiles[i,j-1].find_word(), self.tiles[i,j+1].find_word())
                            self.rules += [rule]

                    #up -> down
                    if i-1 >= 0 and i+1 <=self.rows-1:
                        if self.tiles[i-1,j].find_word() in self.prefixes and self.tiles[i+1,j].find_word() in self.suffixes:
                            rule = Rule(self.tiles[i-1,j].find_word(), self.tiles[i+1,j].find_word())
                            self.rules += [rule]


    def move_up(self):
        pass
    def move_down(self):
        pass
    def move_left(self):
        pass
    def move_right(self):
        pass


    def push_objects(self, direction, col, row):
        pass


    def check_win(self):
        pass
    def check_lose(self):
        pass

    
    def reset_game(self):
        pass

    def load_map_level(self, file_name):
        pass

### Rule

In [158]:
class Rule:
    def __init__(self, first, second):         #first + is + second 
        self.first = first
        self.second = second
    
    def __repr__(self):
        return self.first +" is "+ self.second


### Object

In [159]:
class Object:
    def __init__(self, property='no bug here'):
        self.property = property

    def interaction(self, another_object:object):
        
        pass
        

In [160]:
class Baba(Object):        
    def interaction(self, another_object:object):
        
        pass

In [161]:
class Rock(Object):
    def interaction(self, another_object:object):
        
        pass
        

In [162]:
class Wall(Object):
    def interaction(self, another_object:object):
        
        pass
        

In [163]:
class Water(Object):
    def interaction(self, another_object:object):
        
        pass
     

In [164]:
class Skull(Object):
    def interaction(self, another_object:object):
        
        pass
        

In [165]:
class Flag(Object):
    def interaction(self, another_object:object):
        
        pass
        

In [166]:
class Word(Object):
    def __init__(self, value):        
        super().__init__('push')
        self.value = value

    def interaction(self, another_object:object):
        
        pass
        

### Tile

In [167]:
class Tile:
    def __init__(self):
        self.objects = np.array([], dtype=object)
    def add_object(self, obj : Object):
        self.objects = np.append(self.objects, [obj])

    def find_word(self):
        for obj in self.objects:
            if isinstance(obj, Word):
                return obj.value
            return ''


### Unit Testing

In [176]:
test_gp = Gameplay(6,15)
test_gp.tiles[4,9].add_object(Word("is"))
test_gp.tiles[4,9].add_object(Flag())
test_gp.tiles[4,9].add_object(Word("is"))
test_gp.tiles[3,9].add_object(Word("baba"))
test_gp.tiles[5,9].add_object(Word("you"))
test_gp.tiles[4,9].add_object(Word("is"))
test_gp.tiles[4,8].add_object(Word("rock"))
test_gp.tiles[4,10].add_object(Word("push"))
test_gp.tiles[5,9].add_object(Word("is"))
test_gp.tiles[5,8].add_object(Word("baba"))
test_gp.tiles[5,10].add_object(Word("rock"))
test_gp.get_rules()

yes


In [172]:
test_tile = Tile()
test_tile

In [177]:
print(test_gp.rules)
print(test_gp.rules[0].first)
print(test_gp.rules[0].second)
print(test_gp.rules[0])

[rock is push, baba is you]
rock
push
rock is push
